# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) の位置が他と重複しない契約者の 2016 年投資額 (tiv_2016) を合計し、**小数 2 桁**に丸めて 1 行で出力せよ。`
* 入力 DF: `Insurance(pid: int, tiv_2015: float, tiv_2016: float, lat: float (NOT NULL), lon: float (NOT NULL))`
* 出力: `tiv_2016`（合計、**2 桁丸め**、1 行 1 列）

## 2) 実装（指定シグネチャ厳守）

> 原則は **列最小化 → グループ処理（transform） → 条件抽出 → 合計 → 丸め**。全量ソート不要、`value_counts`/`transform('size')` を使い、ハッシュ集計で線形近似。

```python
import pandas as pd
import numpy as np

def solve_insurance(insurance: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        insurance (pd.DataFrame): 列は pid, tiv_2015, tiv_2016, lat, lon
    Returns:
        pd.DataFrame: 列名と順序は ['tiv_2016'] （1 行 1 列, 小数 2 桁丸め）
    """
    # 列最小化（必要列のみを扱う）※ビュー的スライスでコスト最小化
    df = insurance[['tiv_2015', 'tiv_2016', 'lat', 'lon']]

    # 条件1: 同一 tiv_2015 が 2 件以上
    cnt_tiv2015 = df.groupby('tiv_2015', sort=False)['tiv_2015'].transform('size')
    mask_dup_tiv2015 = cnt_tiv2015 > 1

    # 条件2: (lat, lon) がユニーク（= 1 件のみ）
    cnt_city = df.groupby(['lat', 'lon'], sort=False)['lat'].transform('size')
    mask_unique_city = cnt_city == 1

    # 両条件を満たす行のみ合計
    total = df.loc[mask_dup_tiv2015 & mask_unique_city, 'tiv_2016'].sum()

    # 小数 2 桁に丸め（表示では末尾ゼロは落ちるが、要件は数値的丸め）
    out = pd.DataFrame({'tiv_2016': [float(np.round(total, 2))]})
    return out
```

* 依存 API: `groupby(..., sort=False).transform('size')` は **全量ソート不要**で、内部はハッシュ集計相当（データ分布に依存）。
* 返却は **列名と順序を厳守**して `DataFrame` を 1 行で返す。

## 3) アルゴリズム説明

* `groupby.transform('size')` … グループサイズを元行の形で返す（**セミジョイン**相当のフラグ作成に最適）
* `mask_dup_tiv2015` … `tiv_2015` が **2 件以上**の行を True
* `mask_unique_city` … `(lat, lon)` の組が **1 件のみ**の行を True
* `sum → round(2)` の順に実行（**行ごと丸め → 合算**は要件と異なる）
* **NULL / 重複 / 型**:

  * 本問題は `lat`, `lon` が NOT NULL 前提のため、同値判定をそのまま使用
  * `float` の丸めは `np.round(..., 2)` を使用（表示上の桁は DataFrame のレンダラ依存）

## 4) 計算量（概算）

* `groupby.transform('size')` ×2: **O(N)** 近辺（ハッシュ集約前提、キー分布依存）
* マスクでの抽出・合計: **O(N)**
* 追加メモリ: 変換列 2 本 + マスク 2 本で **O(N)**

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム Insurance] --> B[列最小化]
  B --> C["groupby.transform('size') で件数付与"]
  C --> D["条件: tiv_2015>1 かつ (lat,lon)=1"]
  D --> E[該当行を合計]
  E --> F[小数2桁に丸めて返却]
  F --> G["出力 列:tiv_2016(1行)"]
```


# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `同一の tiv_2015 を持つ保険契約者が 1 名以上存在し、かつ (lat, lon) の位置が他と重複しない契約者の 2016 年投資額 (tiv_2016) を合計し、**小数 2 桁**に丸めて 1 行で出力せよ。`
* 入力 DF: `Insurance(pid: int, tiv_2015: float, tiv_2016: float, lat: float (NOT NULL), lon: float (NOT NULL))`
* 出力: `tiv_2016`（合計、**2 桁丸め**、1 行 1 列）

## 2) 実装（指定シグネチャ厳守）

> **高速化ポイント**: `groupby.transform('size')` をやめ、**`duplicated` のブーリアン生成**だけで条件マスクを作成。
> これにより **全量ソートや中間 Series の常駐を回避**し、メモリと時間を削減できます（ハッシュベースで線形近似）。

```python
import pandas as pd
import numpy as np

def solve_insurance(insurance: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        insurance (pd.DataFrame): 列は pid, tiv_2015, tiv_2016, lat, lon
    Returns:
        pd.DataFrame: 列名と順序は ['tiv_2016'] （1 行 1 列, 小数 2 桁丸め）
    """
    # 列最小化（ビュー）
    df = insurance[['tiv_2015', 'tiv_2016', 'lat', 'lon']]

    # 条件1: tiv_2015 が複数行に出現（= 重複あり）
    mask_dup_tiv2015 = df['tiv_2015'].duplicated(keep=False)

    # 条件2: (lat, lon) がユニーク（= どこにも重複しない）
    # duplicated(keep=False) は重複行すべて True → ユニークは ~(...).
    mask_unique_city = ~df[['lat', 'lon']].duplicated(keep=False)

    # 該当行の tiv_2016 を合計 → 2 桁丸め
    total = df.loc[mask_dup_tiv2015 & mask_unique_city, 'tiv_2016'].sum()
    return pd.DataFrame({'tiv_2016': [float(np.round(total, 2))]})

# Analyze Complexity
# Runtime 302 ms
# Beats 88.46%
# Memory 67.65 MB
# Beats 62.13%

```

### 代替（`value_counts` ベース・分布が極端に偏る場合向け）

> キー種類が極端に少なく **同一値が巨大に偏る**場合は `value_counts` がさらに効くケースがあります（ただしタプル生成がメモリコスト）。

```python
def solve_insurance_vc(insurance: pd.DataFrame) -> pd.DataFrame:
    df = insurance[['tiv_2015', 'tiv_2016', 'lat', 'lon']]

    # tiv_2015 の個数を map（1 パス）
    cnt_tiv = df['tiv_2015'].value_counts()
    mask_dup_tiv2015 = df['tiv_2015'].map(cnt_tiv).gt(1)

    # (lat, lon) の個数を value_counts（2 パス）
    cnt_city = df.value_counts(subset=['lat', 'lon'])
    # 各行の (lat,lon) → 個数を引くために Series[tuple] に投影
    mask_unique_city = df[['lat', 'lon']].apply(tuple, axis=1).map(cnt_city).eq(1)

    total = df.loc[mask_dup_tiv2015 & mask_unique_city, 'tiv_2016'].sum()
    return pd.DataFrame({'tiv_2016': [float(np.round(total, 2))]})

# Analyze Complexity
# Runtime 398 ms
# Beats 26.48%
# Memory 68.57 MB
# Beats 21.45%

```

## 3) アルゴリズム説明

* 使用 API

  * `Series.duplicated(keep=False)` … **重複がある要素を一括検出**（True = 重複出現すべて）
  * `DataFrame.duplicated(subset, keep=False)` … **複合キー**の重複検出
  * `sum` → `np.round(..., 2)` … 合計→丸めの順序で要件満たす
* **NULL / 重複 / 型**

  * 問題前提で `lat`, `lon` は **NOT NULL**。浮動小数の完全一致で比較。
  * `duplicated` は **ハッシュベース**でグループサイズ算出より軽量（中間列を作らない）。
  * 丸めは表示用ではなく **値として 2 桁に丸め**た `float` を返却。

## 4) 計算量（概算）

* `duplicated`（1 列 / 複合キー）: **O(N)** 近辺（ハッシュ）
* マスク結合・合計: **O(N)**
* 追加メモリ: **ブール配列 2 本** + スライスのみ（`transform('size')` 版より軽い）

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム Insurance] --> B[列最小化]
  B --> C[duplicated で重複/ユニーク判定]
  C --> D["条件: tiv_2015 重複 かつ (lat,lon) ユニーク"]
  D --> E[該当行の tiv_2016 を合計]
  E --> F[小数2桁に丸めて返却]
  F --> G["出力 列:tiv_2016(1行)"]
```

---

### 追加の微調整ヒント

* **列型の一貫性**: `tiv_2015`, `lat`, `lon` の dtype が `float64` で混在していないか確認（`float32` 混在はハッシュの変換コストが増えることあり）。
  必要なら前処理で `astype({'tiv_2015':'float64','lat':'float64','lon':'float64'})` を一度だけ実施。
* **コピー抑制**: 外側で `insurance` を毎回作り直している場合は、本関数内での列スライスを**最後まで使い切る**（中間 DataFrame を増やさない）。
* **極端なスパース/密分布**: キーの種類が極端に少なければ `value_counts` 版（上）も計測対象に。環境により勝ち負けが入れ替わります。


`duplicated` を使ったブール合成で **線形** に落とせていて、実務でも通用する形になっています。さらに数％〜十数％を狙える微調整案をまとめます（可読性をあまり落とさない範囲）。

## ちょい速化・省メモリのコツ

1. **`to_numpy(copy=False)` で配列化して合計**
   `Series.sum()` も速いですが、最終合計は NumPy に落とすとわずかに有利なことが多いです。
   （インデックスアライン不要・型確定・ギャザー1回）

2. **dtype を明示してハッシュコストを安定化**
   `float32/float64` が混在すると内部変換が走ることがあります。`astype` を**一度だけ**使って `float64` に寄せると安定。

3. **中間 Series を減らす（列参照の一時変数）**
   同じ列を何度も取り出さない。参照をローカル変数に束縛して JIT 的な最適化が効くケースがある。

4. **（任意）大規模で `(lat, lon)` の重複が極少なら**
   `duplicated` は十分速いですが、極端にユニークが多い場合は `value_counts` よりも `duplicated` のままが最速なことが多いので、今のままでOKです。

## マイクロ最適化版

```python
import pandas as pd
import numpy as np

def solve_insurance(insurance: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        insurance (pd.DataFrame): 列は pid, tiv_2015, tiv_2016, lat, lon
    Returns:
        pd.DataFrame: 列名と順序は ['tiv_2016'] （1 行 1 列, 小数 2 桁丸め）
    """
    # 必要列だけビュー取得 → dtype を一度だけ明示（混在時の内部変換を回避）
    df = insurance[['tiv_2015', 'tiv_2016', 'lat', 'lon']].astype({
        'tiv_2015': 'float64',
        'tiv_2016': 'float64',
        'lat': 'float64',
        'lon': 'float64',
    }, copy=False)

    tiv2015 = df['tiv_2015']
    latlon   = df[['lat', 'lon']]
    tiv2016  = df['tiv_2016']

    # 条件1: tiv_2015 が重複（=同値が2件以上）
    mask_dup_tiv2015 = tiv2015.duplicated(keep=False)

    # 条件2: (lat, lon) がユニーク（=1件のみ）
    mask_unique_city = ~latlon.duplicated(keep=False)

    # 合成マスクで抽出し、NumPy で合計 → 2桁丸め
    sel = mask_dup_tiv2015 & mask_unique_city
    total = np.add.reduce(tiv2016.to_numpy(copy=False)[sel.to_numpy(copy=False)])
    return pd.DataFrame({'tiv_2016': [float(np.round(total, 2))]})
```

### 補足

* **I/O・ソート禁止**要件も満たしています（`sort_values` 不使用）。
* `np.add.reduce(...)` は `sum()` 同等ですが、ブロードキャストを避けた**一次元連結の合計**に最適化された実装で、`np.sum(...)` よりわずかに速いことがあります（環境依存）。
* 丸めは **合計→丸め** の順序を維持。

この程度のタッチでも、データ量や dtype の混在具合によっては **5〜15%** ほどの改善が出ることがあります。今のコードが十分速い環境なら、そのまま採用でOKです！
